In [ ]:
using Pkg
Pkg.activate(".")

## Loading Relevant Packages

In [ ]:
using PyPlot
using EmpHCA
using LinearAlgebra
using Random
using Turing, MCMCChains, Distributions
using DataFrames
using CSV
using JLD2
using AdvancedMH
using HypothesisTests

import StatsPlots

PyPlot.svg(true)
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["svg.fonttype"] = "none"
rcParams["pdf.fonttype"] = 42

## Loading behavioral data of Experiment 1

In [ ]:
Path_Load = "data/Experiment1/clean/"

# exclusion information
ExcDF = DataFrame(CSV.File(Path_Load * "ExclusionInfo.csv"))
# data of room selection
dataDF = DataFrame(CSV.File(Path_Load * "SelectionData.csv"))
# choosing subject ids of non-excluded participants
subjectIDs = ExcDF.subject[ExcDF.task_outliers .== 0]

dataDF


Let's look at the data of participant 1:

In [ ]:
i_sub = 1
df = dataDF[dataDF.subject .== i_sub, :]
# removing timed-out trials
df = df[df.timeout .== false, :]

In [ ]:
# Xinds: pairs of rooms indices shown to the participant 
# see src/Functions_for_gold.jl for the correspondance
Xinds = [[df.room1[i], df.room2[i]] .+ 1 for i = 1:size(df)[1]]
Xinds'

In [ ]:
# as: sequence of rooms chosen by the participant
as = df.action .+ 1;
as'

## Loading room information

Information of different rooms to be used for inference

In [ ]:
Prooms, ΔState, ΔStateDict = gold_proom_sets();
N_rooms = length(Prooms); Ymax = 1; Xmax = 1;

# transition probabilities for the center node in Room 2
# Prooms[2][a,sp] is the probabilities of going to state sp after taking action a
Prooms[2]

In [ ]:
# mapping of sp to physical location
ΔStateDict

## Non-hierarchical model-selection

This part perform the inference (for a single subject) corresponding to the generative model described in Methods (Eq. 7).
On a normal computer, the inference takes around 3-5 minutes.

In [ ]:
# MCMC sampling hyperparameters
n_chains = 3
chain_lenght = 2500
burn_in_lenght = 500
sample_lenght = 10

# transforming Xinds to the sequence of transition probabilities
Xs = gold_Room2X_indexbased(Prooms, Xinds, ΔState, ΔStateDict, Xmax, Ymax,1)
# transforming Xinds to the sequence of number of actions
Nas = [[size(Prooms[x[1]])[1],size(Prooms[x[2]])[1]] for x = Xinds]

# defining the generative model (see src/Functions_for_goldDataE1.jl)
modelAll = TuringGoldBasicInfvsEmplvsNa(Xs, Xinds, Nas, as; N_rooms = N_rooms, K = 1)
# defining the sampling method for MCMC (see Turing.jl tutorials)
gAll = Gibbs(HMC(0.01, 50, :θ, :l, :β, :βa, :βθ), MH(:m, :γ))
# inference
chnAll = sample(modelAll, gAll,  MCMCThreads(), chain_lenght, n_chains);

In [ ]:
# primary plotting the MCMC results
StatsPlots.plot(chnAll; legend=true)

In [ ]:
# cleaning up the MCMC samples by accounting for the burn-in length and thinning lenght
chnAll_df = DataFrame(chnAll)
filter!(row -> row.iteration > burn_in_lenght, chnAll_df)
chnAll_df = chnAll_df[1:sample_lenght:size(chnAll_df)[1],:]

In [ ]:
# plotting the processed results
x = 1:4
x_names = ["Random","N-Act","Emp-l","General"]
fig = figure(); ax = subplot(1,1,1)
ax.bar(x,[mean(chnAll_df.m .== i) for i = x])
ax.set_ylim([0,1]); ax.set_xlim([0,x[end]+1]); 
ax.set_xticks(x); ax.set_xticklabels(x_names,rotation = 90)
ax.set_ylabel("P(Model | Data)")
display(fig)